In [ ]:
import pandas as pd
import numpy as np
import re
from pprint import pprint
import os

from google.colab import files

from google.colab import drive
drive.mount('/content/drive')
os.chdir("/content/drive/My Drive")

df = pd.read_csv('mbti_preprocessed.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### MBTI 분류 / 불용어 처리

In [ ]:
mbti_E = df[(df['type_label']==0) | (df['type_label']==1) | (df['type_label']==2) | (df['type_label']== 3) | (df['type_label']==4) | (df['type_label']==5) | (df['type_label']== 6) | (df['type_label']==7)]
mbti_I = df[(df['type_label']==8) | (df['type_label']==9) | (df['type_label']==10) | (df['type_label']== 11) | (df['type_label']== 12) | (df['type_label']== 13) | (df['type_label']== 14) | (df['type_label']==15)]

In [ ]:
mbti_E.to_csv('mbti_E.csv', index=False)
mbti_I.to_csv('mbti_I.csv', index=False)

In [ ]:
from nltk.corpus import stopwords
stop = ['tapatalk', 'iphone', 'ha', 'using', 'enfjs', 'thats', 'kitteh', 'th', 'ya', 'alot', 'fucking', 'favourite', 'dont', 'sp', 'sx', 'dom', 'cant', 'esfjs', 'estps', 'enfps', 'entps', 'nf', 'sent', 'inferior', 'ive', 'score','particularly', 'youtube', 'husband', 'sensor', 'coffee', 'yea', 'idk', 'summer', 'fourm', 'welcome', 'much', 'istps', 'hahaha', 'wanna', 'istps']
mbti_E['posts'] = mbti_E['posts'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
mbti_I['posts'] = mbti_I['posts'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


### DTM 생성

In [ ]:
#MBTI_E
from sklearn.feature_extraction.text import CountVectorizer
vec_E = CountVectorizer(max_df=0.1, max_features=1000, min_df=2, ngram_range=(1,2))
    
document_term_matrix_E = vec_E.fit_transform(mbti_E['posts'])      
DTM_E = pd.DataFrame(document_term_matrix_E.toarray(), columns=vec_E.get_feature_names())  

#MBTI_I
vec_I = CountVectorizer(max_df=0.1, max_features=1000, min_df=2, ngram_range=(1,2))
document_term_matrix_I = vec_I.fit_transform(mbti_I['posts'])     
DTM_I = pd.DataFrame(document_term_matrix_I.toarray(), columns=vec_I.get_feature_names())  

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


### TF-IDF 값 구하기

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer
transformer = TfidfTransformer()
transformer

MBTI_E_TFIDF = transformer.fit_transform(DTM_E)
print('#shape of tfidf matrix:', MBTI_E_TFIDF)

MBTI_I_TFIDF = transformer.fit_transform(DTM_I)
print('#shape of tfidf matrix:', MBTI_I_TFIDF)

#shape of tfidf matrix:   (0, 986)	0.10252930226199639
  (0, 956)	0.09678225255817992
  (0, 932)	0.08658214310361312
  (0, 929)	0.08726042955975216
  (0, 927)	0.10252930226199639
  (0, 914)	0.20811559155086307
  (0, 904)	0.10653365834523411
  (0, 903)	0.10458795489869989
  (0, 880)	0.10228307392191793
  (0, 868)	0.10711743337068165
  (0, 866)	0.09094604673138092
  (0, 854)	0.08852784655014731
  (0, 841)	0.09468774238283247
  (0, 812)	0.18687550740528164
  (0, 775)	0.09001264468678027
  (0, 756)	0.0869869860308209
  (0, 749)	0.0873982383465293
  (0, 744)	0.20505860452399277
  (0, 725)	0.08970881788544532
  (0, 689)	0.08781610626403508
  (0, 680)	0.09468774238283247
  (0, 678)	0.10568188054810294
  (0, 677)	0.09191424185484257
  (0, 647)	0.09542998988957839
  (0, 634)	0.1032823084747913
  :	:
  (1998, 341)	0.09981622809261836
  (1998, 326)	0.10288135501104761
  (1998, 303)	0.21539904978892632
  (1998, 298)	0.10311437880330891
  (1998, 289)	0.0903411625085476
  (1998, 257)	0.0984063349609

In [ ]:
MBTI_E_TFIDF_DF = pd.DataFrame(MBTI_E_TFIDF.toarray(), columns=vec_E.get_feature_names())
MBTI_E_TFIDF_DF

MBTI_I_TFIDF_DF = pd.DataFrame(MBTI_I_TFIDF.toarray(), columns=vec_I.get_feature_names())
MBTI_I_TFIDF_DF

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,absolute,abstract,accepted,according,account,achieve,acquaintance,act like,acting,active,...,would never,would probably,would think,would want,writer,yay,yep,yes yes,yup,zone
0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.124138,0.0,0.000000,0.000000
1,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.142293,0.000000
2,0.120617,0.0,0.126036,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000
3,0.000000,0.0,0.000000,0.081103,0.0,0.0,0.0,0.0,0.0,0.0,...,0.087509,0.0,0.0,0.092112,0.089496,0.0,0.000000,0.0,0.000000,0.000000
4,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.104643
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6671,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.112034,0.000000
6672,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.167775
6673,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.125134,0.000000
6674,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000


In [ ]:
MBTI_E_TFIDF_SUM = MBTI_E_TFIDF_DF.sum()
MBTI_E_TFIDF_SUM.sort_values(ascending=False)[300:350]


build           15.744349
reminds         15.742259
every time      15.738164
like know       15.728643
felt like       15.725876
including       15.725056
excuse          15.714947
aggressive      15.708641
attempt         15.707522
attack          15.706179
core            15.702816
flirting        15.694888
passion         15.686693
tomorrow        15.684562
green           15.671672
method          15.659259
drama           15.648952
strength        15.645715
regardless      15.631257
understood      15.628926
experienced     15.615320
caught          15.600182
health          15.597022
judge           15.594228
bar             15.575096
board           15.556656
pop             15.554169
king            15.553043
twice           15.546023
various         15.533418
think people    15.533169
suggestion      15.503819
discus          15.498033
yay             15.493256
prove           15.483650
yup             15.472713
appear          15.469638
among           15.440791
everyday    

In [ ]:
MBTI_I_TFIDF_SUM = MBTI_I_TFIDF_DF.sum() 
MBTI_I_TFIDF_SUM.sort_values(ascending=False)[300:350]

guilty            53.080357
fiction           52.990756
waste             52.880693
helping           52.796343
would probably    52.729218
apart             52.709358
happening         52.605857
active            52.484169
unique            52.446079
feel way          52.443518
passion           52.436986
plenty            52.389850
private           52.385138
created           52.367282
secret            52.313500
hopefully         52.277653
iq                52.274814
appear            52.265571
rational          52.264709
marriage          52.258022
hm                52.229536
loving            52.201296
scene             52.189693
explanation       52.171600
smell             52.168953
join              52.167934
attack            52.149978
shirt             52.138955
pet               52.071504
distance          52.052578
twice             52.029712
hmmm              52.025797
opportunity       52.005533
every day         51.999037
total             51.972139
excuse            51

### LDA분석

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation

document_term_matrix_E
print('CountVectorizer Shape:', document_term_matrix_E.shape)

lda = LatentDirichletAllocation(n_components=2)
lda.fit(document_term_matrix_E)

def display_topics(model, feature_names, num_top_words):
    for topic_index, topic in enumerate(model.components_):
        print('Topic #', topic_index)

        topic_word_indexes = topic.argsort()[::-1]
        top_indexes=topic_word_indexes[:num_top_words]
        
        feature_concat = ' '.join([feature_names[i] for i in top_indexes])                
        print(feature_concat)

feature_names = vec_E.get_feature_names()

display_topics(lda, feature_names, 10)

CountVectorizer Shape: (1999, 1000)
Topic # 0
wife drug depression sharing challenge company communication psychology position student
Topic # 1
socionics color blushed profile mbti type shocked mad anyways join bunch


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
document_term_matrix_I
print('CountVectorizer Shape:', document_term_matrix_I.shape)

lda_I = LatentDirichletAllocation(n_components=2)
lda_I.fit(document_term_matrix_I)

feature_names = vec_I.get_feature_names()

display_topics(lda_I, feature_names, 10)

CountVectorizer Shape: (6676, 1000)
Topic # 0
war city law religious drug universe debate philosophy american star
Topic # 1
girlfriend crush close friend unhealthy helped sharing upset scared typed conflict


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
